In [2]:
# Importing the dependencies.
import pandas as pd
import numpy as np
from citipy import citipy
from config import weather_api_key
import requests
import time
from datetime import datetime

In [3]:
# Creating array of sets of random latitude and longitude
lats = np.random.uniform(low=-90.000,high=90.000, size = 1500)
lngs = np.random.uniform(low=-180.000,high=180.000, size = 1500)
latLngs = zip(lats,lngs)
latLngs

In [4]:
# Adding the latitudes & longitudes to a list
coordinates = list(latLngs)

In [5]:
#Creating list of cities
cities = []

#Identifying the nearest city for each lat and long combo
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0],coordinate[1]).city_name

    #Checking to see if the city is unique. If so, it is added to the "cities" list.
    if city not in cities:
        cities.append(city)

#print the city count to confirm there are enough
len(cities)

615

In [6]:
url = 'https://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=' + weather_api_key
#Creating an empty list to hold the weather data
cityData = []

#Printing the beginning of the logging
print("Beginning Data Retrieval      ")
print('------------------------------')

#Creating counters
recordCount = 1
setCount = 1

#looping through all the cities in our list
for i, city in enumerate(cities):
    #grouping cities into sets of 50 for logging purposes
    if i % 50 == 0 and i >= 50:
        setCount += 1
        recordCount = 1
        time.sleep(60)

    #creating endpoint URL with each city
    cityURL = url + '&q=' + city.replace(' ','+')

    #logging the URL, Record and set numbers and the city
    print(f'Processing Record {recordCount} of Set {setCount} | {city}')
    #adding 1 to record count
    recordCount += 1
# Run an API request for each of the cities.
    try:
        #parsing the json and retrieving the data
        cityWeather = requests.get(cityURL).json()

        #parsing out the needed data
        cityLat = cityWeather['coord']['lat']
        cityLng = cityWeather['coord']['lon']
        cityMaxTemp = cityWeather['main']['temp_max']
        cityHumidity = cityWeather['main']['humidity']
        cityClouds = cityWeather['clouds']['all']
        cityWind = cityWeather['wind']['speed']
        cityCountry = cityWeather['sys']['country']

        #Converting date to ISO standard
        cityDate = datetime.utcfromtimestamp(cityWeather['dt']).strftime('%Y-%m-%d %H:%M:%S')

        #Apprending the city information into cityData list
        cityData.append({'City': city.title(),
                         'Lat':cityLat,
                         'Lng':cityLng,
                         'Max Temp':cityMaxTemp,
                         'Humidity':cityHumidity,
                         'Cloudiness':cityClouds,
                         'Wind Speed':cityWind,
                         'Country':cityCountry,
                         'Date':cityDate})
# If an error is experienced, skip the city.
    except:
        print('City not found. Skipping City...')
        pass
    #Indicate that data loading is complete
print('--------------------------')
print('Data Retrieval Complete!')
print('--------------------------')

Beginning Data Retrieval      
------------------------------
Processing Record 1 of Set 1 | shimoda
Processing Record 2 of Set 1 | saskylakh
Processing Record 3 of Set 1 | chokurdakh
Processing Record 4 of Set 1 | busselton
Processing Record 5 of Set 1 | santa cruz
Processing Record 6 of Set 1 | rikitea
Processing Record 7 of Set 1 | akdepe
Processing Record 8 of Set 1 | bluff
Processing Record 9 of Set 1 | cidreira
Processing Record 10 of Set 1 | hobart
Processing Record 11 of Set 1 | wagga wagga
Processing Record 12 of Set 1 | khatanga
Processing Record 13 of Set 1 | qaanaaq
Processing Record 14 of Set 1 | bengkulu
Processing Record 15 of Set 1 | balaipungut
Processing Record 16 of Set 1 | port alfred
Processing Record 17 of Set 1 | atuona
Processing Record 18 of Set 1 | tiksi
Processing Record 19 of Set 1 | thompson
Processing Record 20 of Set 1 | vanimo
Processing Record 21 of Set 1 | ushuaia
Processing Record 22 of Set 1 | burdur
Processing Record 23 of Set 1 | bolungarvik
City n

In [7]:
len(cityData)

566

In [9]:
#Converting the array of dictionaries to a pandas DataFrame
cityDataDf = pd.DataFrame(cityData)

#Reordering the columns
newColumnOrder = ['City', 'Country', 'Date', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness', 'Wind Speed']
cityDataDf = cityDataDf[newColumnOrder]

cityDataDf.head()

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Shimoda,JP,2021-12-02 05:02:31,34.6667,138.9500,54.10,61,29,10.96
1,Saskylakh,RU,2021-12-02 05:02:32,71.9167,114.0833,-36.35,100,97,1.03
2,Chokurdakh,RU,2021-12-02 05:02:32,70.6333,147.9167,-38.16,100,91,5.84
3,Busselton,AU,2021-12-02 05:02:33,-33.6500,115.3333,76.08,38,4,4.45
4,Santa Cruz,BO,2021-12-02 05:00:13,-17.8000,-63.1667,72.54,96,97,7.92


In [10]:
# Creating the output file (CSV)
outputDataFile = 'weather_data/cities.csv'
#Exporting the cityData into a CSV
cityDataDf.to_csv(outputDataFile, index_label='City_ID')